# Mini Problem

**The problem and data is referred from coursera's "Introduction to Data Science in Python" course assignment**

In this problem, files of metropolitan regions and associated sports teams is given. The aim is to find out the correlation between win/loss ratio's and population of the city. Here data of NHL(hockey) in 2018 is utilised. The analysis includes data mining, processing and cleaning. Code is developed using python libraries in jupyter platform.
The correlation is calculated using [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html) function.
For cities data follow this link [`wikipedia_data`](https://drive.google.com/file/d/1HyfwbtUOIYS3HK8XGMWxG_v8X7NCrg0G/view?usp=sharing)
And for game data follow this [`link`](https://drive.google.com/file/d/1AKfThopH8pxUeYir5Perj1qT9e8LzqPO/view?usp=sharing)

In [16]:
#importing Necessary libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [17]:
#importing datasets
nhl_df=pd.read_csv("nhl.csv")
#
#
cities = pd.read_html("wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]# extraction of necessary data

#small preview of data
cities.head(10)

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


### Solution formulation Function

In [18]:
# YOUR CODE HERE
#Setting coloumn names
cities.columns = ['Metropolitan area','Population','NFL','MLB','NBA','NHL']
    
#Extracting required data from cities dataframe
df1 = cities[['Metropolitan area','Population','NHL']]
    
# Removing unwanted the contents in 'NFL','MLB','NBA' and 'NHL' columns
df1['NHL'] = df1['NHL'].str.extract('([\w ]*)')
# 
#Modifyig the entries to form good data handling
df1['NHL'].replace({'RangersIslandersDevils':'Rangers,Islanders,Devils','KingsDucks':'Kings,Ducks'}, inplace=True)
    
df1 = df1.assign(NHL = df1.NHL.str.split(",")).explode('NHL')
df1 = df1[df1['NHL'].str.contains("^[\w].*") == True]
#from nhl_df dataframe
df_nhl = nhl_df[nhl_df['year'] == 2018][['team','W','L',]]
df_nhl = df_nhl[df_nhl['W'].str.isnumeric() == True]
    
#modifying the given dta frame to conduct operations
df_nhl['Population'] = None
df_nhl['Metropolitan area'] = None
for team in df1['NHL'].unique():
    idx = df_nhl[df_nhl['team'].str.contains(team)].index[0]
    df_nhl['Population'].loc[idx] = df1[df1['NHL'] == team]['Population'].values[0]
    df_nhl['Metropolitan area'].loc[idx] = df1[df1['NHL'] == team]['Metropolitan area'].values[0]
df_nhl['W/L ratio'] = pd.to_numeric(df_nhl['W'])/(pd.to_numeric(df_nhl['W']) + pd.to_numeric(df_nhl['L']))
df_nhl['Population'] = pd.to_numeric(df_nhl['Population'])
    
#Final dtaframe
final_df = df_nhl.groupby('Metropolitan area').mean()
        
#raise NotImplementedError()
    
population_by_region = final_df['Population'] # pass in metropolitan area population from cities
win_loss_by_region = final_df['W/L ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
ans = stats.pearsonr(population_by_region, win_loss_by_region)[0]

#Display of Answer
print(ans)

0.012486162921209881


C:\Users\ARJUN K VIKRAM\miniconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\ARJUN K VIKRAM\miniconda3\lib\site-packages\pandas\core\series.py:4569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
